### Load library and data

In [ ]:
library(RColorBrewer)
library(Seurat)
library(presto)
library(dplyr)
library(dittoSeq)
library(devtools)
library(SeuratData)
library(Connectome)
library(EnhancedVolcano)
library(monocle)
library(tibble)
library(cowplot)
library(ggplot2)
library(gplots)
library(plyr)
library(grid)
library(pheatmap)
library(RColorBrewer)
library(ggpubr)


### Color

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
display.brewer.all()
c1 =brewer.pal(n = 8, name = "Set1")
c2 =brewer.pal(n = 8, name = "Dark2")
c3 =brewer.pal(n = 8, name = "Paired")
c4 =brewer.pal(n = 8, name = "Accent")

Cols = c(c1,c2, c3, c4)

In [ ]:
post_5cluster <- LoadH5Seurat("/home/sujin/NORA_post_enr_5clusters_raw.h5seurat", assays='RNA')

In [ ]:
post_5cluster <- NormalizeData(NORA, normalization.method = "LogNormalize", scale.factor = 10000)
post_5cluster <- FindVariableFeatures(NORA, selection.method = "vst", nfeatures = 2000)
all.genes <- rownames(post_5cluster)
post_5cluster <- ScaleData(post_5cluster, features = all.genes)

In [ ]:
cluster_order <- c("EP8-VIM-Pre","EP8-VIM-Post","EP9-AGER-Pre","EP9-AGER-Post","FB6-CD74-Pre","FB6-CD74-Post","EC9-LYZ-Pre","EC9-LYZ-Post","MAC1-SELENOP-Pre","MAC1-SELENOP-Post")
post_5cluster$Annotation <- factor(post_5cluster$Annotation, levels = cluster_order)
Idents(post_5cluster) <- post_5cluster$Annotation

In [ ]:
group3_genes<- c("SCGB1A1","SLPI","MT-CO2","MT-CO3","PPDPF", "TSC22D1","ATF3","TIMP3","FBP1","NR4A1","ZNF331","DUSP2","IL1B","NFKBIZ","IER3","GADD45B","HLA-DRA","HLA-DRB1","IFI30","PSAP","NFKBIA","FTL")

In [ ]:
DotPlot(post_5cluster, features = group3_genes) + RotatedAxis() + theme(axis.text.x = element_text(angle = 90))+scale_colour_gradient2(low = "blue", mid = "white", high = "red") 

### Proportion (box plot)

In [ ]:
prop_nora <- read.csv("/home/sujin/NORA_total_cluster_proportion.csv")
prop_nora$Treatment <- factor(prop_nora$Treatment , levels = c("Pre","Post"))
prop_nora$EGFR_mutation <- factor(prop_nora$EGFR_mutation , levels = c("E19del", "L858R"))
prop_nora$MPR_type <- factor(prop_nora$MPR_type , levels = c("MPR", "nMPR" ))
prop_nora$Clinical_response <- factor(prop_nora$Clinical_response , levels = c( "PR" ,"SD"))

In [ ]:

p<-ggplot(prop_nora, aes(x=Treatment, y=Epithelial.cells, fill=Treatment)) +
  stat_boxplot(geom = "errorbar", width = 0.25) +
  geom_boxplot(outlier.shape = NA)
p + geom_dotplot(binaxis='y', stackdir='center', dotsize=1.1) + 
  stat_compare_means(comparisons = list(c("Pre","Post")),
                     label = "p.adj", method = "wilcox", size = 4.1, test.args=list(alternative = "two.sided", var.equal = TRUE, paired=FALSE)) +
  scale_fill_manual(values=c("#ABB99F", "#3D7E54")) +
  theme_linedraw() +
  theme_light() +
  ylim(0, 0.725) +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_blank()) + 
  theme(axis.title=element_text(size=16)) + 
  xlab(" ") + ylab("Epithelial cells") + 
  theme_classic() +
  theme(axis.title.x= element_text (size=12, color="Black",face = "bold"),
        axis.title.y= element_text (size=12, color="Black",face = "bold"),
        axis.text.x= element_text (size=12, color="Black"),
        axis.text.y= element_text (size=12, color="Black")) +
  theme(legend.position="none")



### Trajectory (monocle2)

In [ ]:

matrix_to_expression_df<- function(x, obj){
  df<- x %>%
    as.matrix() %>%
    as.data.frame() %>%
    tibble::rownames_to_column(var= "gene") %>%
    tidyr::pivot_longer(cols = -1, names_to = "barcode", values_to = "expression") %>%
    tidyr::pivot_wider(names_from = "gene", values_from = expression) %>%
    left_join(obj@meta.data %>%
                tibble::rownames_to_column(var = "barcode"))
  return(df)
}


get_expression_data<- function(obj, assay = "RNA", slot = "data",
                               genes = NULL, cells = NULL){
  if (is.null(genes) & !is.null(cells)){
    df<- GetAssayData(obj, assay = assay, slot = slot)[, cells, drop = FALSE] %>%
      matrix_to_expression_df(obj = obj)
  } else if (!is.null(genes) & is.null(cells)){
    df <- GetAssayData(obj, assay = assay, slot = slot)[genes, , drop = FALSE] %>%
      matrix_to_expression_df(obj = obj)
  } else if (is.null(genes & is.null(cells))){
    df <- GetAssayData(obj, assay = assay, slot = slot)[, , drop = FALSE] %>%
      matrix_to_expression_df(obj = obj)
  } else {
    df<- GetAssayData(obj, assay = assay, slot = slot)[genes, cells, drop = FALSE] %>%
      matrix_to_expression_df(obj = obj)
  }
  return(df)
}

In [ ]:
expression_data <- get_expression_data(NORA, genes = genes)
expression_data

In [ ]:
monocle_AT1 <- newCellDataSet(data_select,
                               phenoData = pd_select,
                               featureData = fd_select,
                               lowerDetectionLimit = 0.5,
                               expressionFamily = negbinomial.size())

In [ ]:
monocle_at1_select <- monocle_AT1

monocle_at1_select <- estimateSizeFactors(monocle_at1_select)
monocle_at1_select <- estimateDispersions(monocle_at1_select)

monocle_at1_select <- detectGenes(monocle_at1_select, min_expr = 1)

head(fData)
head(pData)


In [ ]:
# standardise to Z-distribution
x <- pd_select@data$n_genes
x_1 <- (x - mean(x)) / sd(x)
summary(x_1)

In [ ]:
df <- data.frame(x = x_1)
ggplot(df, aes(x)) +
  geom_histogram(bins = 50) +
  geom_vline(xintercept = c(-2, 2), linetype = "dotted", color = 'red')

In [ ]:
ggplot(pd_select@data, aes(n_genes, cell)) + geom_point()

In [ ]:
disp_table <- dispersionTable(monocle_apCAF_cp)
head(disp_table)

In [ ]:
table(disp_table$mean_expression>=0.01)

In [ ]:
unsup_clustering_genes <- subset(disp_table, mean_expression >= 0.01)

In [ ]:
monocle_at1_select <- setOrderingFilter(monocle_at1_select, unsup_clustering_genes$gene_id)
plot_ordering_genes(monocle_at1_select)

In [ ]:
monocle_at1_select <- reduceDimension(monocle_at1_select, max_components = 2, reduction_method = 'DDRTree', verbose = TRUE)

In [ ]:
plot_cell_trajectory(monocle_apCAF_cp_select_10, color_by = "State", cell_size = 0.8, show_branch_points = FALSE, cell_link_size = 1) +theme_bw(base_size = 15)

In [ ]:
plot_cell_trajectory(monocle_apCAF_cp_select_10, color_by = "TKI", cell_size = 0.8, show_branch_points = FALSE, cell_link_size = 1) +theme_bw(base_size = 15) 

In [ ]:
plot_cell_trajectory(monocle_apCAF_cp_select_10, color_by = "Pseudotime", cell_size = 0.8, show_branch_points = FALSE, cell_link_size = 1) +theme_bw(base_size = 15) 